! Note, that first two code cells take several hours to execute, avoid running them.

Preprocess each project separately, to turn it into a dataset for a trained model:

In [ ]:
import os


for project_name in os.listdir('sources'):
    os.system("bash preprocess.sh " + project_name)

To calculate quality metrics, evaluate a trained model passing each preprocessed project as test data:

In [1]:
import csv
import os

result_file = "quality_metrics.csv"
header = ["Project", "F1", "Precision", "Recall"]
with open(result_file, "w") as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    f.close()
for project_name in os.listdir('sources'):
    os.system("python3 code2seq.py --project %s --load models/java-large-model/model_iter52.release --test "
              "data/%s/%s.test.c2s" % (project_name, project_name, project_name))

Let's look at the projects, where recognition results were best and worst:

In [4]:
import pandas as pd

data = pd.read_csv(result_file)
data = data.sort_values(by="F1")
data

,Project,F1,Precision,Recall
3,socketio__socket.io-client-java,0.279433,0.305426,0.257516
77,rest-assured__rest-assured,0.327217,0.437792,0.261236
32,EnterpriseQualityCoding__FizzBuzzEnterpriseEdi...,0.379221,0.392473,0.366834
57,libgdx__packr,0.389381,0.448980,0.343750
60,square__keywhiz,0.396026,0.422675,0.372539
...,...,...,...,...
48,hugeterry__CoordinatorTabLayout,0.855072,0.867647,0.842857
72,daimajia__NumberProgressBar,0.867925,0.877863,0.858209
71,Trinea__android-auto-scroll-view-pager,0.878378,0.902778,0.855263
89,Tencent__MSEC,0.904964,0.910945,0.899061


Now, find a mean and spread for each of quality metrics:

In [6]:
import numpy as np

def get_mean_and_spread(metric):
    print(metric+":", "mean", np.mean(data[metric]), "spread", np.std(data[metric]))

get_mean_and_spread("F1")
get_mean_and_spread("Precision")
get_mean_and_spread("Recall")

F1: mean 0.6487602105263157 spread 0.13753456932061955
Precision: mean 0.6802209578947367 spread 0.12633969306886522
Recall: mean 0.6226689368421053 spread 0.1487102951523929
